In [ ]:
import numpy

In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torch.utils.data as data
# import torchvision
# import torchvision.models as models
# from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle

from functions import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from model import FineTuneModel

In [3]:

## set path ##
data_path = "./ucf_image/"    # define UCF-101 spatial data path
action_name_path = "./UCF101actions.pkl"  # load preprocessed action names
save_model_path = "./record/"  # save Pytorch models

## gpu setting ##
# GPU_NUM, GPU_NUM2 = 0,  2
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# device2 = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU
# print('Current cuda device ', torch.cuda.current_device())


# ## training parameters ##

# epochs = 15
# batch_size = 3
# learning_rate = 1e-4
# log_interval = 10
# img_x, img_y = 240, 320  # resize video 2d frame size

# ## model parameter ##
# dim = 100
# k = 101                 # number of target category

# ## Loader parameter ##
# params = {'batch_size': batch_size, 'shuffle': True}
# ## load UCF101 actions names ##
# with open(action_name_path, 'rb') as f:
#     action_names = pickle.load(f)   # load UCF101 actions names    
# action_names[58] = 'HandStandPushups' # fix the label name to match the folder name

# ## convert labels -> category ##
# le = LabelEncoder()
# le.fit(action_names)
# action_category = le.transform(action_names).reshape(-1, 1)
# enc = OneHotEncoder()
# enc.fit(action_category)
# actions = []
# fnames = os.listdir(data_path)
# all_names = []
# for f in fnames:
#     loc1 = f.find('v_')
#     loc2 = f.find('_g')
#     actions.append(f[(loc1 + 2): loc2]) # 저장되어 있는 파일 순서대로 labeling

#     all_names.append(f)
    
# ## list all data files ##
# all_X_list = all_names              # all video file names
# all_y_list = labels2cat(le, actions)    # all video labels

# ## train, test split ##
# train_list, test_list, train_label, test_label = train_test_split(all_X_list, all_y_list, test_size=0.25, random_state=42)

# ## total 25 frames (min size) ##
# begin_frame, end_frame, skip_frame = 1, 26, 1 
# selected_frames = np.arange(begin_frame, end_frame, skip_frame).tolist()
# print("The total Number of selected frame is", len(selected_frames))


# ## image transformation ##
# transform = transforms.Compose([transforms.Resize([img_x, img_y]),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=[0.5], std=[0.5])])

# ## UCF101 30 images dataset ##
# train_set, valid_set = Dataset_3DCNN(data_path, train_list, train_label, selected_frames, transform=transform), \
#                        Dataset_3DCNN(data_path, test_list, test_label, selected_frames, transform=transform)


# train_loader = data.DataLoader(train_set, **params)
# valid_loader = data.DataLoader(valid_set, **params)


In [5]:
dim = 100
k = 101
learning_rate = 1e-4

model = FineTuneModel(dim, k)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


model_path = './record/bin/FT_epoch1.pth'
optim_path = './record/bin/FT_optimizer_epoch1.pth'

load_model = torch.load(model_path)
load_optim = torch.load(optim_path)
model.load_state_dict(load_model)
optimizer.load_state_dict(load_optim)


Get Pre-trained Model: vgg16


In [7]:
model

FineTuneModel(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [8]:
for x, y in train_loader:
    y_pred = []
    for i, x_in in enumerate(x):
        print(i)
        y_pred.append(model(x_in))
    break

In [18]:
for i in range(3):
    print(torch.argmax(y_pred[i]).item(), y[i].item())

37 37
1 1
85 85


In [19]:
y.shape

torch.Size([3, 1])

In [25]:
import numpy as np

np.load('./record/FTmodel_epoch_training_scores50.npy')

array([[0.        , 0.33333333, 0.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.66666667, ..., 1.        , 1.        ,
        1.        ],
       [0.66666667, 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [ ]:
optim_path

In [29]:
check_point = torch.save({
    'model': model.state_dict(),
    'optimizer':optimizer.state_dict()
}, 'temp.pth')

In [32]:
temp = torch.load('temp.pth')
temp['model']
optimizer.load_state_dict(temp['optimizer'])

In [33]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)

In [11]:
class UCF_DNN(nn.Module):
    def __init__(self, h_in, h1, h2, h3, h4, h5, h_out):
        super(UCF_DNN, self).__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(h_in,h1),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            
            nn.Linear(h1,h2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            
            nn.Linear(h2,h3),
            nn.ReLU(),
            
            nn.Linear(h3,h4),
            nn.Dropout2d(0.3),
            nn.ReLU(),
            
            nn.Linear(h4,h_out)
        )
        
    def forward(self, x):
        x = x.reshape(-1, 3000) # T * D 
        x = self.fc(x)
        return x

In [12]:
import numpy as np
import torch
# from model import *

In [22]:
x = np.random.rand(1,30,100)
x = torch.from_numpy(x).float()
h_in, h_out = x.shape[1]*x.shape[2]*1, 101 # UCF101
h1, h2, h3, h4, h5 = 512, 256, 128, 128, 100 
model = UCF_DNN(h_in, h1, h2, h3, h4, h5, h_out)
model.

<generator object Module.named_buffers at 0x7fd3099a3b50>

In [18]:
model(x)

tensor([[ 0.0531,  0.0224, -0.0790, -0.0024,  0.0144, -0.0745,  0.0532,  0.0710,
          0.0454,  0.0302, -0.0307,  0.0020,  0.0172, -0.0959, -0.1060,  0.0261,
          0.0544,  0.0692,  0.0448, -0.0510,  0.0498,  0.0439, -0.0296, -0.0382,
         -0.0523,  0.0278, -0.0664,  0.0452,  0.0415,  0.0789, -0.0530,  0.0390,
          0.0185, -0.0289,  0.0497,  0.0985,  0.0486, -0.0710, -0.0391,  0.0169,
          0.0712, -0.0707, -0.1000,  0.0084, -0.0218, -0.0904,  0.0815,  0.1067,
          0.0292,  0.0391, -0.0792, -0.0743,  0.0194,  0.0786,  0.0310,  0.0038,
         -0.0448,  0.0006,  0.0829, -0.0061,  0.1112, -0.0627, -0.0279, -0.0282,
         -0.0243,  0.0651,  0.0384,  0.0059, -0.0749,  0.1212, -0.0924, -0.0488,
         -0.0269, -0.0759, -0.0449, -0.0616,  0.0267,  0.0133,  0.0639,  0.0734,
         -0.0142,  0.0243,  0.0773,  0.0502,  0.0371,  0.0505, -0.0336,  0.0541,
         -0.0254,  0.0166,  0.1035,  0.0070, -0.0716, -0.0819,  0.0651,  0.1369,
          0.0101,  0.0108,  

In [3]:
model

NameError: name 'model' is not defined